In [2]:
import PySimpleGUI as sg
import modulo as m
import os

BD = []

# ------------------------------------- JANELA COM TABELA -------------------------------------

def atoresEfilmes(bd):
    data_headings=['Actor/Actress', 'Title: id']
    listaDeAtores=m.ordenaGenerosAtores(bd,'cast')
    listaFinal=[]
    Titulos=" "
    for conjunto in listaDeAtores:
        Nova=[]
        ator, titulos=conjunto                 #desfaz o tuplo criado e devolvido pela funçao ordenaAtores
        Nova.append(ator)                      #adicionamos à lista nova o ator para que este seja o primeiro elemento da lista
        Titulos=Titulos+titulos
        Nova.append(Titulos)                   #adicionamos à lista nova a string que contem os todos os titulos
        listaFinal.append(Nova)                #adicionamos à lista listaFinal a lista nova
        Titulos=""
    return(listaFinal, data_headings)      #devolvemos a listaFinal (que é uma lista de listas, necessáro para construir a table)  

def janelatabela(lista, headings):    
    layout4=[[sg.Table(values=lista,font = "Arial,12", headings=headings,
                            vertical_scroll_only = False,
                            auto_size_columns=True,
                            key='-TABLE-',
                            max_col_width = 50,
                            justification = "left",
                            num_rows=20,
                            size=(20,22),
                            background_color='#1A6E74')], 
                            #sg.Button("OK",font = "Arial,12")
                            ]
    window4= sg.Window(title="Tabela",background_color='#1A6E74', resizable= True, default_element_size=(20,1)).Layout(layout4)

    stop=False  
    while not stop:
        event, values = window4.read()
        if event == sg.WIN_CLOSED or event == "OK":
            stop = True

    window4.close()
    
def generosEfilmes(bd):
    data_headings=['Genres', 'Title: id']
    listaDeGeneros=m.ordenaGenerosAtores(bd,'genres')
    listaFinal=[]
    Titulos=" "
    for conjunto in listaDeGeneros:
        Nova=[]
        genero, titulos=conjunto                 #desfaz o tuplo criado e devolvido pela funçao ordenaAtores
        Nova.append(genero)                      #adicionamos à lista nova o ator para que este seja o primeiro elemento da lista
        Titulos=Titulos+titulos
        Nova.append(Titulos)                   #adicionamos à lista nova a string que contem os todos os titulos
        listaFinal.append(Nova)                #adicionamos à lista listaFinal a lista nova
        Titulos=""
    return(listaFinal, data_headings)
# ------------------------------------- JANELA EXTRA -------------------------------------

def janelaExtra(evento):
    if evento=="Ator":
        exemplo="Ex: \"Selena Gomez\""           #Para as pessoas saberem o exemplo a utilizar
        procura="Insira o nome do ator que pretende procurar:"
    elif evento=="Género":
        exemplo="Ex: \"Action\""
        procura="Insira o género que pretende procurar:"
    elif evento=="Título":
        exemplo="Ex: \"Dog\""
        procura="Insira uma palavra que pretende procurar:"
    elif evento =='ID':
        exemplo = 'Ex: \'232\''
        procura ='Insira o ID que pretende procurar'
    layout2=[[sg.Text(procura,background_color='#1A6E74',font = "Arial,12")], [sg.InputText(exemplo, key='-PROCURA-',font = "Arial,12"), sg.Button("Pesquisar" ,button_color=('white', '#6ABDC3'),font = "Arial,12")]]
    window2= sg.Window(title="Janela de pesquisa",background_color='#1A6E74', default_element_size=(25,1)).Layout(layout2)

    stop=False  
    while not stop:
        event, values = window2.read()
        if event == sg.WIN_CLOSED or event == "Pesquisar":
            stop = True

    window2.close()
    return values["-PROCURA-"]                # DEVOLVE A PESQUISA QUE O UTILIZADOR QUER EFETUAR



# ------------------------------------- MENSAGEM DE ERRO -------------------------------------

def janelaErro(mensagem):
    layout=[[sg.Text(mensagem,background_color='#1A6E74',font = "Arial,12")], [sg.Button("Ok", button_color=('white', '#6ABDC3'),font = "Arial,12")]]
    window= sg.Window(title="Mensagem",background_color='#1A6E74', default_element_size=(15,1)).Layout(layout)

    stop=False  
    while not stop:
        event, values = window.read()
        if event == "Ok" or event == sg.WIN_CLOSED:
            stop = True

    window.close()

# ------------------------------------- INSERIR NOVO FILME  -------------------------------------

def novoFilme(e):
    layout3=[[sg.Text("Titulo:",background_color='#1A6E74'), sg.InputText('', key='-TITULO-', size=(32))],
            [sg.Text("Ano:",background_color='#1A6E74'), sg.InputText('', key='-ANO-', size=(33))],
            [sg.Text("Elenco:",background_color='#1A6E74'), sg.InputText('', key='-ELENCO-', size=(31))],
            [sg.Text("Género:",background_color='#1A6E74'), sg.InputText('', key='-GÉNERO-', size=(30))],
            [sg.Button("Adicionar", button_color=('white', '#6ABDC3'))]]
            

    window3= sg.Window(title=e, background_color='#1A6E74', default_element_size=(30)).Layout(layout3)

    stop=False  
    while not stop:
        event, values = window3.read()
        if event == "Adicionar" or event == sg.WIN_CLOSED:
            stop = True

    window3.close()
    filme = [values['-TITULO-'],values['-ANO-'],values['-ELENCO-'],values['-GÉNERO-']]
    if m.detNone(filme):
        return 
    else:
        return(filme)


# ------------------------------------- INTERFACE PRINCIPAL -------------------------------------

listbox=[]
layout = [[sg.Column([[sg.Button("     Carregar BD     ", button_color=('white', '#6ABDC3'),tooltip = "Introduza o nome da base de dados que pretende carregar na caixa de texto abaixo",font = "Arial,12"), 
                       sg.Button("     Guardar BD     ", button_color=('white', '#6ABDC3'),tooltip = "Introduza o nome com que pretende guardar a base de dados na caixa de texto abaixo, \n preferencialmente sem extensão (será sempre guardado em json)",font = "Arial,12")],
            [sg.InputText(''"", key='-DADOS-', size=(34,1),font = "Arial,12")],
            [sg.Button("Listar filmes alfabeticamente", tooltip = "Listagem dos títulos existentes na base de dados, alfabeticamente",button_color=('white', '#6ABDC3'),font = "Arial,12")],
            [sg.Text("Consultar por:",font = "Arial,12", background_color='#1A6E74'), sg.Button("Género", tooltip = "Carregue para pesquisar um género e listar todos os filmes desse género", button_color=('white', '#6ABDC3'),font = "Arial,12"),
            sg.Button("Ator", tooltip = "Carregue para pesquisar um ator e listar todos os seus filmes",button_color=('white', '#6ABDC3'),font = "Arial,12"),
            sg.Button("Título", tooltip = "Carregue para pesquisar uma palavra e listar todos os filmes que a contenham",button_color=('white', '#6ABDC3'),font = "Arial,12"),
            sg.Button("ID", tooltip = "Carregue para procurar um filme pelo seu ID",button_color=('white', '#6ABDC3'),font = "Arial,12")],
            [sg.Button("Filmes na BD", tooltip = "Carregue para saber quantos filmes existem na base de dados carregada",button_color=('white', '#6ABDC3'),font = "Arial,12")],
            [sg.Button("Inserir novo filme", tooltip = "Carregue para adicionar um novo filme à base de dados",button_color=('white', '#6ABDC3'),font = "Arial,12"),
            sg.Button("Alterar registo", tooltip = "Altere um registo através do ID",button_color=('white', '#6ABDC3'),font = "Arial,12")], 
            [sg.Button("Distribuição por género", tooltip = "Carregue para saber quais as distribuições por género \n dos filmes existentes na base de dados",button_color=('white', '#6ABDC3'),font = "Arial,12")],
            [sg.Button("Distribuição por ator",  tooltip = "Carregue para saber quais as distribuições por ator \n dos filmes existentes na base de dados", button_color=('white', '#6ABDC3'),font = "Arial,12")],
            [sg.Button("Atores ordenados", tooltip = "Carregue para ordenar alfabeticamente os atores existentes \n na base de dados, os respetivos filmes e id", button_color=('white', '#6ABDC3'),font = "Arial,12")],
            [sg.Button("Generos ordenados", tooltip = "Carregue para ordenar alfabeticamente os generos existentes \n na base de dados, os respetivos filmes e id", button_color=('white', '#6ABDC3'),font = "Arial,12")],
            [sg.Button("Sair", button_color=('white', '#6ABDC3'),font = "Arial,12")]], background_color='#1A6E74',scrollable=True,size=(400,430)), sg.Listbox(values=listbox, size=(40,17),font = "Arial,12", key='-LIST-', horizontal_scroll=True)]]



window=sg.Window(title="Cinemateca", resizable=True, background_color='#1A6E74').Layout(layout)
stop=False  
while not stop:
    event, values = window.read()

    if event=="     Carregar BD     ":
        if os.path.exists(values['-DADOS-']):         # VERIFICA SE O FICHEIRO EXISTE  #BFE9A2
            BD= m.lerficheiro(values['-DADOS-'])
            if BD==None:
                janelaErro("Introduza um ficheiro com extensão válida!") 
            else:
                janelaErro("BD carregada com sucesso!")
                m.id(BD)
        else:
            janelaErro("Não foi encontrada nenhuma base de dados com esse nome!")

    elif event=="     Guardar BD     ":
        if values['-DADOS-'].replace(" ", "")== '':              # verifica se o utilizador nao inseriu apenas espaços
            janelaErro("Introduza um nome para a nova BD!") 
        elif os.path.isfile(values['-DADOS-']):                  # verifica se o utilizador inseriu o nome de um ficheiro ja existente
            janelaErro("Já existe um ficheiro com esse nome!") 
        else:
            values['-DADOS-'] = m.tiraext(values['-DADOS-'])
            m.guardarficheiro(values['-DADOS-'],BD)

    elif event=="Listar filmes alfabeticamente":
        if BD != []:
            lista= m.formatL(BD)
            lista.sort()
            top=['Title |  Year  |  Cast  |   Genres  |  ID  ']
            window['-LIST-'].Update(values=top+lista)
        else:
            janelaErro("Nenhuma BD carregada")
        

    elif event=="Ator" or event=="Género" or event=="Título" or event =='ID':
        if BD != []:
            pesquisa=janelaExtra(event)                   # FAZER CÓDIGO QUE VAI TRABALHAR COM A PALAVRA INSERIDA
            if pesquisa!= None :
                if event=="Ator":                  # E DEVOLVER A RESPOSTA NA LIST BOX 
                    listbox=m.listarAG(BD, pesquisa,'cast')
                    res=m.formatacao(listbox)
                    window['-LIST-'].Update(values=res)
                if event=="Género":
                    listbox=m.listarAG(BD,pesquisa,'genres')
                    res=m.formatacao(listbox)
                    window['-LIST-'].Update(values=res)
                if event =='Título':
                    listbox = m.formatacao(m.consultarfilme(BD,pesquisa))
                    window['-LIST-'].Update(values=listbox)
                if event == 'ID':
                    a = m.procuraID(pesquisa,BD)
                    if a == None:
                        janelaErro('Não existe nenhum filme com o ID fornecido!')
                    else:
                        listbox = m.formatacao([a])
                        window['-LIST-'].Update(values=listbox)
        else:
            janelaErro("Nenhuma BD carregada")

    elif event=="Filmes na BD":
        listbox=m.contaFilmes(BD)
        window['-LIST-'].Update(values=listbox)

    elif event=="Inserir novo filme":
        infFilme = novoFilme(event)
        if infFilme!=None:
            infFilme[2] = infFilme[2].split(',')
            infFilme[3] = infFilme[3].split(',')
            novo = m.novofilme(infFilme)
            BD.append(novo)
            m.id(BD)

    elif event =='Alterar registo':
        pesquisa = janelaExtra('ID')
        a = m.procuraID(pesquisa,BD)
        if a == None:
            janelaErro('Não existe nenhum filme com o ID fornecido!')
        else:
            infFilme = novoFilme(event)
            if infFilme!=None:
                infFilme[2] = infFilme[2].split(',')
                infFilme[3] = infFilme[3].split(',')
                novo = m.novofilme(infFilme)
                m.eliminaent(BD,pesquisa)
                BD.append(novo)
                m.id(BD)

    
    elif event=="Distribuição por género":
        if BD != []:
            listbox=m.dictolist(m.distribuicao(BD,'genres'))
            window['-LIST-'].Update(values=listbox)
        else:
            janelaErro("Nenhuma BD carregada")
    elif event=="Distribuição por ator":
        if BD!=[]:
            listbox=m.dictolist(m.distribuicao(BD,'cast'))
            window['-LIST-'].Update(values=listbox)
        else:
            janelaErro("Nenhuma BD carregada")
    elif event=="Atores ordenados":
        if BD!=[]:
            listadeatores, topo=atoresEfilmes(BD)
            janelatabela(listadeatores, topo)
        else:
            janelaErro("Nenhuma BD carregada")
    elif event=="Generos ordenados":
        if BD!=[]:
            listadegeneros, topo=generosEfilmes(BD)
            janelatabela(listadegeneros, topo)
        else:
            janelaErro("Nenhuma BD carregada")
        
    if event == "Sair" or event == sg.WIN_CLOSED:
        stop = True

window.close()


